# Treemix Ag1000G phase2
For build a dataset Treemix I need unlinked SNPs. So I have to prune my allele count datasets to obtain SNPs in high LD.
For doing this I need:
    - Phase2 Genotype callset
    - Phase2 Allele count
    - Outgroup Allele count

In this notebook I edited an old Alistair's notebook of the Phase1 of Ag1000G (<b>20151001 treemix prep 4</b>). On my phase2 datasets I have already the biallelic allele counts so I skipped the searching and filtering for biallelic SNPs

Import my modules:

In [1]:
%run imports.ipynb

imports_20150407.ipynb:26: MatplotlibDeprecationWarning: 
The savefig.jpeg_quality rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  "outputs_hidden": false


Import callsets:

In [2]:
callset_pass= callset_biallel
allele_counts= zarr.open('../data/phase2_biallel_allele_count.zarr/')
outgroup_allele_counts= zarr.open('../data/outgroup_alleles_phase2.zarr/')

------------------------

Define functions to locate biallelic allele counts on a range for my outgroup and phase2 datasets:

In [3]:
def outgroup_ascertainment(chrom, start, stop, outgroups):
    
    # locate region
    pos = allel.SortedIndex(callset_pass[chrom]['variants']['POS'][:])
    locr = pos.locate_range(start, stop)
    
    # ascertain SNPs
    loca = np.zeros(pos.shape, dtype='b1')
    loca[locr] = True
    log('outgroup ascertainment, initial', nnz(loca))
    for s in outgroups:
        ac = allel.AlleleCountsArray(outgroup_allele_counts[chrom][s][:])
        # non-missing
        locs = (ac.sum(axis=1) > 0)
        loca &= locs
        log(s, nnz(loca))
        
    return loca
        

In [4]:
def ingroup_ascertainment(chrom, start, stop, segpops):

    # locate region
    pos = allel.SortedIndex(callset_pass[chrom]['variants']['POS'][:])
    locr = pos.locate_range(start, stop)

    # ascertain SNPs
    loca = np.zeros(pos.shape, dtype='b1')
    loca[locr] = True
    log('ingroup ascertainment, initial', nnz(loca))

    
    # require segregating
    for pop in segpops:
        ac = allel.AlleleCountsArray(allele_counts[chrom][pop][:])
        loc_seg = ac.min(axis=1) > 0
        loca &= loc_seg
        log('after require segregating in', pop, nnz(loca))
        
    return loca

Define function for ld pruning. LD-pruning remove SNPs with an high correlation. Using windows this function compute pairwise LD between all SNPs within each window, then removing one SNP from each correlated pair.

Define function for generating treemix file:

In [5]:
def to_treemix(acs, fn):
    pops = sorted(acs.keys())
    n_variants = acs[pops[0]].shape[0]
    n_alleles = acs[pops[0]].shape[1]
    assert n_alleles == 2, 'only biallelic variants supported'
    for pop in pops[1:]:
        assert n_variants == acs[pop].shape[0], 'bad number of variants for pop %s' % pop
        assert n_alleles == acs[pop].shape[1], 'bad number of alleles for pop %s' % pop
        
    with open(fn, 'wt', encoding='ascii') as f:
        print(' '.join(pops), file=f)
        for i in range(n_variants):
            print(' '.join([','.join(map(str, acs[pop][i])) for pop in pops]), file=f)


Define a new function that randomly downsample if I have a large dataset and applies ld-pruning on it:

In [6]:
def downsample_and_prune(chrom, start, stop, loc_asc,
                         n=100000, ldp_size=500, ldp_step=250, ldp_threshold=.1, ldp_n_iter=1):

    # all variant positions
    pos = allel.SortedIndex(callset_pass[chrom]['variants']['POS'][:])
    posa = pos[loc_asc]

    # randomly downsample
    if n < posa.shape[0]:
        posds = np.random.choice(posa, n, replace=False)
        posds.sort()
        posds = allel.SortedIndex(posds)
    else:
        # skip downsampling
        posds = posa
    locds = pos.locate_keys(posds)    

    # load genotype data
    genotype = allel.GenotypeChunkedArray(callset_pass[chrom]['calldata/GT'])
    geno_subset = genotype.subset(sel0=loc_asc)
    gn = geno_subset.to_n_alt()

    
    # prune    
    for i in range(ldp_n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=ldp_size, step=ldp_step, threshold=ldp_threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        log('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gnu = gn.compress(loc_unlinked, axis=0)
        posu = pos.compress(loc_unlinked)
        locu = pos.locate_keys(posu)

    return locu

Define last function, the analysis function that includes all function below and applies these on my populations, outgroups, chromosomes of interest.

In [24]:
def run_analysis(rname, chrom, start, stop, outgroups, segpops,
                 n=100000, ldp_size=500, ldp_step=250, ldp_threshold=.1, ldp_n_iter=1):

    # initial ascertainment
    loc_og_asc = outgroup_ascertainment(chrom, start, stop, outgroups=outgroups)
    loc_ig_asc = ingroup_ascertainment(chrom, start, stop, segpops=segpops)
    loc_asc = loc_og_asc & loc_ig_asc
    log('initial ascertainment', nnz(loc_asc))
    
    # downsample and prune
    locu = downsample_and_prune(chrom, start, stop, loc_asc, 
                                n=n, ldp_size=ldp_size, ldp_step=ldp_step, 
                                ldp_threshold=ldp_threshold, ldp_n_iter=ldp_n_iter)
    
    # write allele counts
    acsu = dict()
    for pop in segpops:
        acsu[pop] = allele_counts[chrom][pop][:, :2][locu]
    for pop in outgroups:
        acsu[pop] = outgroup_allele_counts[chrom][pop][:, :2][locu]

    outdir = 'treemix/seg_%s_og_%s_ldp_%s' % ('_'.join(segpops), '_'.join(outgroups), ldp_n_iter)
    !mkdir -pv {outdir}
    fn = os.path.join(outdir, '%s.allele_counts.txt' % rname)
    to_treemix(acsu, fn)
    !gzip -fv {fn}


Declaring values for generating my treemix file and ran on it for chromosome 3R, 3L, X, and the X region involved on speciation between <i>An.gambiae</i> and <i>An.coluzzii</i>

In [25]:
outgroups = ['chri']
segpops = ['BFcol', 'CIcol', 'GHcol', 'GNcol','GHgam', 'BFgam', 'GNgam', 'GM', 'GW']
n = 200000
ldp_n_iter = 1
region_3R_24mbp = '3R-24Mbp', '3R', 1, 24_000_000
region_3L_free = '3L-free', '3L', 18_000_000, 41_000_000

-----------------------------
## Treemix on 24Mbp 3R-free

In [26]:
rname, chrom, start, stop = region_3R_24mbp
log(rname, chrom, start, stop)
run_analysis(rname, chrom, start, stop, outgroups, segpops, n=n, ldp_n_iter=ldp_n_iter)

3R-24Mbp 3R 1 24000000
outgroup ascertainment, initial 5760020
chri 4113161
ingroup ascertainment, initial 5760020
after require segregating in BFcol 1662192
after require segregating in CIcol 935048
after require segregating in GHcol 745633
after require segregating in GNcol 253224
after require segregating in GHgam 193813
after require segregating in BFgam 192709
after require segregating in GNgam 189321
after require segregating in GM 186583
after require segregating in GW 186277
initial ascertainment 120572
iteration 1 retaining 86854 removing 33718 variants
treemix/seg_BFcol_CIcol_GHcol_GNcol_GHgam_BFgam_GNgam_GM_GW_og_chri_ldp_1/3R-24Mbp.allele_counts.txt:	 92.4% -- replaced with treemix/seg_BFcol_CIcol_GHcol_GNcol_GHgam_BFgam_GNgam_GM_GW_og_chri_ldp_1/3R-24Mbp.allele_counts.txt.gz


---------------------------------------
## Treemix on 3L (18 Mbp to 41 Mbp)

In [27]:
rname, chrom, start, stop = region_3L_free
log(rname, chrom, start, stop)
run_analysis(rname, chrom, start, stop, outgroups, segpops, n=n, ldp_n_iter=ldp_n_iter)

3L-free 3L 18000000 41000000
outgroup ascertainment, initial 5358122
chri 3463220
ingroup ascertainment, initial 5358122
after require segregating in BFcol 1554906
after require segregating in CIcol 865263
after require segregating in GHcol 690085
after require segregating in GNcol 234815
after require segregating in GHgam 178327
after require segregating in BFgam 177181
after require segregating in GNgam 174069
after require segregating in GM 171534
after require segregating in GW 171291
initial ascertainment 100081
iteration 1 retaining 72112 removing 27969 variants
treemix/seg_BFcol_CIcol_GHcol_GNcol_GHgam_BFgam_GNgam_GM_GW_og_chri_ldp_1/3L-free.allele_counts.txt:	 92.5% -- replaced with treemix/seg_BFcol_CIcol_GHcol_GNcol_GHgam_BFgam_GNgam_GM_GW_og_chri_ldp_1/3L-free.allele_counts.txt.gz
